# Project forecasting

## Imports

Informations about the data [here](https://transp-or.epfl.ch/documents/technicalReports/CS_LPMC.pdf).

## Functions

## Goal

1. Report the market shares predicted by Modelpref for each scenario.
Do they match your expectations? Compare those with the original market shares. [1 point]

## Scenario 1

An additional charge of £1.5 to the car users

## Scenario 2

A decrease of the public transport cost by 20%.